# Assignment Module 2: Product Classification

The goal of this assignment is to implement a neural network that classifies smartphone pictures of products found in grocery stores. The assignment will be divided into two parts: first, you will be asked to implement from scratch your own neural network for image classification; then, you will fine-tune a pretrained network provided by PyTorch.


## Preliminaries: the dataset

The dataset you will be using contains natural images of products taken with a smartphone camera in different grocery stores:

<p align="center">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Granny-Smith.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Pink-Lady.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Lemon.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Banana.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Vine-Tomato.jpg" width="150">
</p>
<p align="center">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Yellow-Onion.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Green-Bell-Pepper.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Arla-Standard-Milk.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Oatly-Natural-Oatghurt.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Alpro-Fresh-Soy-Milk.jpg" width="150">
</p>

The products belong to the following 43 classes:
```
0.  Apple
1.  Avocado
2.  Banana
3.  Kiwi
4.  Lemon
5.  Lime
6.  Mango
7.  Melon
8.  Nectarine
9.  Orange
10. Papaya
11. Passion-Fruit
12. Peach
13. Pear
14. Pineapple
15. Plum
16. Pomegranate
17. Red-Grapefruit
18. Satsumas
19. Juice
20. Milk
21. Oatghurt
22. Oat-Milk
23. Sour-Cream
24. Sour-Milk
25. Soyghurt
26. Soy-Milk
27. Yoghurt
28. Asparagus
29. Aubergine
30. Cabbage
31. Carrots
32. Cucumber
33. Garlic
34. Ginger
35. Leek
36. Mushroom
37. Onion
38. Pepper
39. Potato
40. Red-Beet
41. Tomato
42. Zucchini
```

The dataset is split into training (`train`), validation (`val`), and test (`test`) set.

The following code cells download the dataset and define a `torch.utils.data.Dataset` class to access it. This `Dataset` class will be the starting point of your assignment: use it in your own code and build everything else around it.

In [3]:
!git clone https://github.com/marcusklasson/GroceryStoreDataset.git

Cloning into 'GroceryStoreDataset'...
Updating files:   6% (393/5717)
Updating files:   7% (401/5717)
Updating files:   8% (458/5717)
Updating files:   9% (515/5717)
Updating files:  10% (572/5717)
Updating files:  11% (629/5717)
Updating files:  11% (641/5717)
Updating files:  12% (687/5717)
Updating files:  13% (744/5717)
Updating files:  14% (801/5717)
Updating files:  15% (858/5717)
Updating files:  15% (903/5717)
Updating files:  16% (915/5717)
Updating files:  17% (972/5717)
Updating files:  18% (1030/5717)
Updating files:  19% (1087/5717)
Updating files:  19% (1127/5717)
Updating files:  20% (1144/5717)
Updating files:  21% (1201/5717)
Updating files:  21% (1253/5717)
Updating files:  22% (1258/5717)
Updating files:  23% (1315/5717)
Updating files:  24% (1373/5717)
Updating files:  24% (1386/5717)
Updating files:  25% (1430/5717)
Updating files:  26% (1487/5717)
Updating files:  27% (1544/5717)
Updating files:  27% (1575/5717)
Updating files:  28% (1601/5717)
Updating files:  29

In [32]:
import numpy as np
from pathlib import Path
from PIL import Image, ImageOps, ImageEnhance
import cv2
from typing import List, Tuple

import torch
from torch import Tensor
from torch.utils.data import Dataset
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
class GroceryStoreDataset(Dataset):

    def __init__(self, split: str, transform=None) -> None:
        super().__init__()

        self.root = Path("GroceryStoreDataset/dataset")
        self.split = split # used for splitting the dataset in train and test
        self.paths, self.labels = self.read_file() #reading the labels and images

        self.transform = transform 

    def __len__(self) -> int: #get the number of elements in the dataset
        return len(self.labels)

    def __getitem__(self, idx) -> Tuple[Tensor, int]: 
        img = Image.open(self.root / self.paths[idx])
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)

        return img, label

    # legge un file di testo che contiene i percorsi delle immagini e le etichette associate
    # restituisce due liste: una con percorsi delle immagini e una con le etichette
    def read_file(self) -> Tuple[List[str], List[int]]: #outputs a tuple
        paths = []
        labels = []

        with open(self.root / f"{self.split}.txt") as f:
            for line in f:
                # path, fine-grained class, coarse-grained class
                path, _, label = line.replace("\n", "").split(", ") #rimuove il carattere di nuova linea, e poi divide in 3 parti con virgola come separatore 
                paths.append(path), labels.append(int(label)) #salva solo percorsi ed etichette

        return paths, labels

    def get_num_classes(self) -> int:
        return max(self.labels) + 1

## Part 1: design your own network

Your goal is to implement a convolutional neural network for image classification and train it on `GroceryStoreDataset`. You should consider yourselves satisfied once you obtain a classification accuracy on the **validation** split of **around 60%**. You are free to achieve that however you want, except for a few rules you must follow:

- You **cannot** simply instantiate an off-the-self PyTorch network. Instead, you must construct your network as a composition of existing PyTorch layers. In more concrete terms, you can use e.g. `torch.nn.Linear`, but you **cannot** use e.g. `torchvision.models.alexnet`.

- Justify every *design choice* you make. Design choices include network architecture, training hyperparameters, and, possibly, dataset preprocessing steps. You can either (i) start from the simplest convolutional network you can think of and add complexity one step at a time, while showing how each step gets you closer to the target ~60%, or (ii) start from a model that is already able to achieve the desired accuracy and show how, by removing some of its components, its performance drops (i.e. an *ablation study*). You can *show* your results/improvements however you want: training plots, console-printed values or tables, or whatever else your heart desires: the clearer, the better.

Don't be too concerned with your network performance: the ~60% is just to give you an idea of when to stop. Keep in mind that a thoroughly justified model with lower accuracy will be rewarded **more** points than a poorly experimentally validated model with higher accuracy.

In [6]:
train_dataset = GroceryStoreDataset(split="train", transform=None)
test_dataset = GroceryStoreDataset(split="test", transform=None)
val_dataset = GroceryStoreDataset(split="val", transform=None)

In [10]:
train_elements = len(train_dataset)
print(f"Number of elements in the train dataset: {train_elements}")

test_elements = len(test_dataset)
print(f"Number of elements in the test dataset: {test_elements}")

val_elements = len(val_dataset)
print(f"Number of elements in the val dataset: {val_elements}")

Number of elements in the train dataset: 2640
Number of elements in the test dataset: 2485
Number of elements in the val dataset: 296


In [26]:
img, label = train_dataset[2040]
print(f"Label: {label}")
img.show()  # Mostra l'immagine

Label: 29


In [28]:
transform = transforms.ToTensor()
img_tensor = transform(img)
print(f"Images shape: ", img_tensor.shape)

Images shape:  torch.Size([3, 348, 348])


Images shape:  torch.Size([3, 348, 348])

- 3 canali = immagine a colori (rgb)
- 348 pixel di altezza
- 348 pixel di larghezza

è un'immagine rgb quadrata con risolutione 348x348 pixel

In [18]:
train_num_classes = train_dataset.get_num_classes()
print(f"Num classes in train dataset: ", train_num_classes) #43 perchè la prima ha index 0

test_num_classes = test_dataset.get_num_classes()
print(f"Num classes in test dataset: ", test_num_classes)

val_num_classes = val_dataset.get_num_classes()
print(f"Num classes in val dataset: ", val_num_classes)

#ogni dataset ha lo stesso numero di classi, lo split è stato fatto corettamente 

Num classes in train dataset:  43
Num classes in test dataset:  43
Num classes in val dataset:  43


In [19]:
paths, labels = train_dataset.read_file()

# Stampare i primi 5 percorsi e le etichette corrispondenti
print("First 5 paths:", paths[:5])
print("First 5 labels:", labels[:5])

First 5 paths: ['train/Fruit/Apple/Golden-Delicious/Golden-Delicious_001.jpg', 'train/Fruit/Apple/Golden-Delicious/Golden-Delicious_002.jpg', 'train/Fruit/Apple/Golden-Delicious/Golden-Delicious_003.jpg', 'train/Fruit/Apple/Golden-Delicious/Golden-Delicious_004.jpg', 'train/Fruit/Apple/Golden-Delicious/Golden-Delicious_005.jpg']
First 5 labels: [0, 0, 0, 0, 0]


### possibile funzione trasform 
Penso questa sia la fare di pre-processing quando si lavora con le immagini
si può fare la stessa cosa con torchvision (ma non ho capito se la possiamo utilizzare o no) oppure con PIL, OpenCV, scikit-image, albumentations

In [ ]:
def pil_transform(image):
    # Ridimensiona l'immagine
    image = image.resize((128, 128))
    
    # Capovolge l'immagine orizzontalmente
    if np.random.rand() > 0.5:
        image = ImageOps.mirror(image)
    
    # Ruota l'immagine di un angolo casuale entro 30 gradi
    angle = np.random.uniform(-30, 30)
    image = image.rotate(angle)
    
    # Modifica casualmente la luminosità
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(np.random.uniform(0.5, 1.5))
    
    # Converte l'immagine in un array numpy e poi in un tensor di PyTorch
    image = np.array(image)
    image = torch.tensor(image).permute(2, 0, 1).float() / 255.0
    
    # Normalizza l'immagine
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    image = (image - mean) / std
    
    return image

In [ ]:
def opencv_transform(image):
    # Convertire l'immagine PIL in un array numpy
    image = np.array(image)
    
    # Ridimensiona l'immagine
    image = cv2.resize(image, (128, 128))
    
    # Capovolge l'immagine orizzontalmente
    if np.random.rand() > 0.5:
        image = cv2.flip(image, 1)
    
    # Ruota l'immagine di un angolo casuale entro 30 gradi
    angle = np.random.uniform(-30, 30)
    M = cv2.getRotationMatrix2D((64, 64), angle, 1)
    image = cv2.warpAffine(image, M, (128, 128))
    
    # Modifica casualmente la luminosità
    alpha = np.random.uniform(0.5, 1.5)
    image = cv2.convertScaleAbs(image, alpha=alpha, beta=0)
    
    # Converte l'immagine in un tensor di PyTorch
    image = torch.tensor(image).permute(2, 0, 1).float() / 255.0
    
    # Normalizza l'immagine
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    image = (image - mean) / std
    
    return image

# Simple CNN 

https://www.digitalocean.com/community/tutorials/writing-cnns-from-scratch-in-pytorch


 The first layer of a CNN is always a convolutional layer. A convolutional layer applies a filter (or kernel) to the input, and passes the result to the next layer. The filter is usually smaller that the image (most often, we can find 3x3 or 5x5 filters), and it moves across the image from top left to bottom right, detecting different features and simplifying the image before passing it onto the next layer. With each convolutional layer, we have to use an activation function. ReLU is a linear function that became the default activation function for many types of neural networks, because it makes the model easy to train, doesn‘t get the vanishing gradient problem compared to Sigmoid and TanH, and often achieves better results.

Conv2D

There are many different types of convolutional layers, but 2D Convolution Layer (Conv2D) is the most common one. The filter "slides" over the 2D input data and multiplies each pixel. As a result, it sums up everything to a single output pixel.

Input shape

As input, CNN takes tensors of shape (image_height, image_width, color_channels). For color images, color channels are RGB (red, green, blue). The format of CIFAR images is (32, 32, 3), so that‘s what we will pass to the CNN.

In [30]:
class CNN(nn.Module):
    def __init__(self, num_classes=43):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(1600, 128) #input di dimensione 1600, 128 dimensione di output 
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [31]:
model = CNN(num_classes=42)

#### hyperparameters

In [ ]:
#loss function
loss_fn = nn.CrossEntropyLoss()

#optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


**kernel_size**

- specifica la dimensione della finestra del filtro che viene applicata all'immagine durante l'operazione di convoluzione
- se kernel_size=3 allora il filtro sarà una matrice 3x3
- un kernel grande cattura più contesto dell'immaigne ma aumenta anche il numero di parametri e il costo computazionale 

**stride**
- specifica di quanti pixel il filtro si sposta o striscia (?) sull'immagine durante l'operazione di convoluzione
- se stride=1 si sposta di un pixel alla volta, se stride=2 di due pixel alla volta e così via 
- un stride maggiore riduce la dimensione spaziale dell'output poichè il filtro copre meno posizioni sull'immagine

**padding**
- aggiunge pixel extra (tipicamente con valore zero) attorno ai bordi dell'immagine prima di applicare la convoluzione
- se padding=1 viene aggiunto un bordo di un pixel attorno all'immagine
- aiuta a preservare la dimensione spaziale dell'immagine dopo la convoluzione, senza la dimensione dell'immagine si riduce ad ogni convoluzione

**MaxPooling (MaxPool2d)**

Il layer di Max Pooling (MaxPool2d in PyTorch) è utilizzato nelle reti neurali convoluzionali (CNN) per ridurre la dimensione spaziale (altezza e larghezza) dell'input (che porta a una riduzione del numero di parametri e del costo computazionale nei layer successivi), mantenendo le caratteristiche più importanti (seleziona il valore massimo in ogni finestra del kernel). Questo processo è noto come "downsampling". Utile anche per la riduzione di overfitting. 
Gli strati di maxpooling tipicamente vengono aggiunti dopo ogni convolutional layer insieme a un layer relu

## Part 2: fine-tune an existing network

Your goal is to fine-tune a pretrained **ResNet-18** model on `GroceryStoreDataset`. Use the implementation provided by PyTorch, do not implement it yourselves! (i.e. exactly what you **could not** do in the first part of the assignment). Specifically, you must use the PyTorch ResNet-18 model pretrained on ImageNet-1K (V1). Divide your fine-tuning into two parts:

1. First, fine-tune the Resnet-18 with the same training hyperparameters you used for your best model in the first part of the assignment.
1. Then, tweak the training hyperparameters in order to increase the accuracy on the validation split of `GroceryStoreDataset`. Justify your choices by analyzing the training plots and/or citing sources that guided you in your decisions (papers, blog posts, YouTube videos, or whatever else you find enlightening). You should consider yourselves satisfied once you obtain a classification accuracy on the **validation** split **between 80 and 90%**.